In [1]:
import pandas as pd

In [2]:
from pathlib import Path

DATA_DIR = Path("data/churn-prediction-25-26")
file_path_dataset = DATA_DIR / "train.parquet"

In [3]:
df_churn = pd.read_parquet(file_path_dataset)

In [4]:
df_churn.columns

Index(['status', 'gender', 'firstName', 'level', 'lastName', 'userId', 'ts',
       'auth', 'page', 'sessionId', 'location', 'itemInSession', 'userAgent',
       'method', 'length', 'song', 'artist', 'time', 'registration'],
      dtype='object')

In [5]:
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17499636 entries, 0 to 25661583
Data columns (total 19 columns):
 #   Column         Dtype         
---  ------         -----         
 0   status         int64         
 1   gender         object        
 2   firstName      object        
 3   level          object        
 4   lastName       object        
 5   userId         object        
 6   ts             int64         
 7   auth           object        
 8   page           object        
 9   sessionId      int64         
 10  location       object        
 11  itemInSession  int64         
 12  userAgent      object        
 13  method         object        
 14  length         float64       
 15  song           object        
 16  artist         object        
 17  time           datetime64[us]
 18  registration   datetime64[us]
dtypes: datetime64[us](2), float64(1), int64(4), object(12)
memory usage: 2.6+ GB


In [6]:
df_churn.describe()

,status,ts,sessionId,itemInSession,length,time,registration
count,1.749964e+07,1.749964e+07,1.749964e+07,1.749964e+07,1.429143e+07,17499636,17499636
mean,2.091387e+02,1.540428e+12,8.480294e+04,1.055937e+02,2.487135e+02,2018-10-25 00:47:01.161927,2018-08-25 04:40:21.543066
min,2.000000e+02,1.538352e+12,1.000000e+00,0.000000e+00,5.220000e-01,2018-10-01 00:00:01,2017-10-14 22:05:25
25%,2.000000e+02,1.539340e+12,2.515900e+04,2.600000e+01,1.998885e+02,2018-10-12 10:33:57.750000,2018-08-10 21:14:59
50%,2.000000e+02,1.540397e+12,7.903800e+04,6.600000e+01,2.340828e+02,2018-10-24 15:58:54,2018-09-05 18:35:50
75%,2.000000e+02,1.541500e+12,1.383680e+05,1.440000e+02,2.768714e+02,2018-11-06 10:25:35,2018-09-20 17:24:57
max,4.040000e+02,1.542672e+12,2.070030e+05,1.426000e+03,3.024666e+03,2018-11-20 00:00:00,2018-11-19 23:34:34
std,3.023050e+01,1.233485e+09,6.141427e+04,1.168854e+02,9.722845e+01,NaN,NaN


In [7]:
# total number of churns in dataset
churn_page ="Cancellation Confirmation"
all_churn_count = (df_churn["page"] == churn_page ).sum()
print(all_churn_count)

4271


In [8]:
# total number of unique users
unique_users = df_churn["userId"].nunique()
print(unique_users)

19140


In [9]:
# number of users who churned from 10.11.2018 to 20.11.2018
end_date = pd.Timestamp("2018-11-10")
churn_after_date = ((df_churn["page"] == churn_page) & (df_churn["time"] >= end_date)).sum()
print(churn_after_date)

667


### Data cleaning

In [10]:
df_churn.sample(5)

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
6212848,200,M,Cuauhtemoc,paid,Jones,1750132,1539959535000,Logged In,NextSong,100115,"St. Louis, MO-IL",16,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",PUT,252.21179,Mourning Air,Portishead,2018-10-19 14:32:15,2018-09-01 14:01:47
24550997,200,M,Abdulmalik,paid,Hayes,1361481,1540922902000,Logged In,NextSong,23967,"Tulsa, OK",16,Mozilla/5.0 (X11; Linux x86_64; rv:31.0) Gecko...,PUT,211.30404,Slamming Doors,Niels van Gogh vs. Eniac,2018-10-30 18:08:22,2018-09-13 15:17:23
7840256,200,F,Fatima,paid,Wilkinson,1824156,1540395263000,Logged In,NextSong,117585,"San Diego-Carlsbad, CA",167,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,227.34322,Up Up & Away,Kid Cudi,2018-10-24 15:34:23,2018-08-15 18:36:41
14156212,200,M,Aiden,paid,Jones,1352609,1542052368000,Logged In,NextSong,173185,"Barnstable Town, MA",113,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,164.17914,Guaranteed,Eddie Vedder,2018-11-12 19:52:48,2018-09-22 04:22:10
20386628,200,M,Izaiah,paid,Barrett,1060841,1539623388000,Logged In,NextSong,9832,"Milwaukee-Waukesha-West Allis, WI",14,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",PUT,236.30322,Society,Eddie Vedder,2018-10-15 17:09:48,2018-08-23 19:57:27


In [11]:
print((df_churn["userId"].str.len() != 7).sum())

0


In [12]:
mask = df_churn["auth"] == "Cancelled"
df_churn[mask]

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
6662491,200,M,Shlok,paid,Johnson,1749042,1540084584000,Cancelled,Cancellation Confirmation,100362,"Dallas-Fort Worth-Arlington, TX",62,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",GET,NaN,None,None,2018-10-21 01:16:24,2018-08-08 13:22:21
9995603,200,M,Andres,paid,Foley,1222580,1540941450000,Cancelled,Cancellation Confirmation,132552,"Watertown, SD",180,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",GET,NaN,None,None,2018-10-30 23:17:30,2018-08-16 02:31:00
15770906,200,M,Azari,paid,Hansen,1385500,1542430449000,Cancelled,Cancellation Confirmation,192519,"Martinsville, VA",189,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",GET,NaN,None,None,2018-11-17 04:54:09,2018-08-16 04:30:35
181774,200,F,Riley,paid,Hernandez,1032628,1538411237000,Cancelled,Cancellation Confirmation,14853,"Reading, PA",591,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",GET,NaN,None,None,2018-10-01 16:27:17,2018-09-22 12:15:45
862670,200,F,Valarie,paid,Moore,1009070,1538587299000,Cancelled,Cancellation Confirmation,29667,"Los Angeles-Long Beach-Anaheim, CA",26,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",GET,NaN,None,None,2018-10-03 17:21:39,2018-09-24 11:58:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23337405,200,M,Ayden,free,Kaufman,1835694,1539096554000,Cancelled,Cancellation Confirmation,1680,"San Francisco-Oakland-Hayward, CA",33,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",GET,NaN,None,None,2018-10-09 14:49:14,2018-08-23 00:07:25
25430665,200,M,Mason,paid,Le,1895668,1542291719000,Cancelled,Cancellation Confirmation,32433,"Salem, OH",414,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",GET,NaN,None,None,2018-11-15 14:21:59,2018-09-20 06:36:02
25268152,200,F,Isabella,free,Smith,1353786,1542064505000,Cancelled,Cancellation Confirmation,28885,"San Francisco-Oakland-Hayward, CA",74,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",GET,NaN,None,None,2018-11-12 23:15:05,2018-08-23 03:24:01
24735863,200,F,Melodi,paid,Baker,1652329,1541179932000,Cancelled,Cancellation Confirmation,16965,"Tulsa, OK",51,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,GET,NaN,None,None,2018-11-02 17:32:12,2018-09-20 12:20:45


In [13]:
cancelled_count = (df_churn["auth"] == "Cancelled" ).sum()
print(cancelled_count)
double_check_auth = ((df_churn["page"] == churn_page) & (df_churn["auth"] == "Cancelled")).sum()
print(double_check_auth)

4271
4271


In [14]:
df_churn['ts'] = pd.to_datetime(df_churn["ts"], unit="ms")

In [15]:
df_churn.sample(5)

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
2916187,200,F,Kalyn,paid,Walker,1468837,2018-10-10 03:06:18,Logged In,NextSong,54167,"San Francisco-Oakland-Hayward, CA",263,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",PUT,169.45587,Oh What A Night,The Dells,2018-10-10 03:06:18,2018-08-08 18:35:47
2623638,200,F,Emberlynn,free,Davis,1045952,2018-10-09 09:34:35,Logged In,Upgrade,38551,"Richmond, VA",38,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",GET,NaN,None,None,2018-10-09 09:34:35,2018-07-29 15:02:17
15577957,200,F,Camila,paid,Carter,1435257,2018-11-16 16:16:57,Logged In,NextSong,177086,"Columbus, OH",66,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",PUT,239.30730,You're The One,Dwight Yoakam,2018-11-16 16:16:57,2018-09-12 23:54:26
3047656,200,F,Madisyn,paid,Murphy,1976657,2018-10-10 12:25:59,Logged In,NextSong,56373,"Jackson, MS",63,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",PUT,129.67138,Sweet Pea,Amos Lee,2018-10-10 12:25:59,2018-09-13 19:20:19
23683063,307,M,Jamie,paid,Thompson,1425700,2018-10-15 17:53:44,Logged In,Thumbs Up,13886,"Los Angeles-Long Beach-Anaheim, CA",73,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,NaN,None,None,2018-10-15 17:53:44,2018-08-16 11:39:54


In [16]:
drop_columns = ["firstName", "lastName", "auth", "method", "ts"]
df_churn.drop(columns=drop_columns, inplace=True)
df_churn['userId'] = df_churn['userId'].astype(int)

In [17]:
df_churn.sample(5)

,status,gender,level,userId,page,sessionId,location,itemInSession,userAgent,length,song,artist,time,registration
11474776,200,M,paid,1931933,NextSong,153893,"Indianapolis-Carmel-Anderson, IN",170,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",258.69016,Chump Change,The New Pornographers,2018-11-04 09:46:01,2018-09-23 12:40:10
15296009,307,F,paid,1689042,Thumbs Up,162850,"Los Angeles-Long Beach-Anaheim, CA",177,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,None,None,2018-11-15 20:27:43,2018-09-21 23:59:41
6068779,200,F,free,1028899,NextSong,98264,"Jacksonville, FL",131,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",147.98322,Mars Attacks,The Misfits,2018-10-19 05:07:47,2018-09-28 07:07:14
14821314,200,M,free,1206739,NextSong,169809,"Chicago-Naperville-Elgin, IL-IN-WI",66,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,304.01261,Song For The Dumped,Ben Folds Five,2018-11-14 14:52:16,2018-09-25 00:54:09
14717327,200,M,paid,1487844,NextSong,160195,"Tulsa, OK",72,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",197.25016,Somebody Told Me,The Killers,2018-11-14 07:48:36,2018-08-02 06:14:22


In [22]:
nan_count = df_churn.isna().sum()

In [23]:
print(nan_count)

status                 0
gender                 0
level                  0
userId                 0
page                   0
sessionId              0
location               0
itemInSession          0
userAgent              0
length           3208203
song             3208203
artist           3208203
time                   0
registration           0
dtype: int64


In [29]:
empty_string_count = (df_churn.select_dtypes(include=["object"]) == "").sum()

In [30]:
print(empty_string_count)

gender       0
level        0
page         0
location     0
userAgent    0
song         0
artist       0
dtype: int64


In [ ]:
mask = df_churn["status"] == 307
df_churn[mask]

,status,gender,level,userId,page,sessionId,location,itemInSession,userAgent,length,song,artist,time,registration
24330,307,M,paid,1749042,Thumbs Up,22683,"Dallas-Fort Worth-Arlington, TX",324,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-10-01 03:10:06,2018-08-08 13:22:21
391536,307,M,paid,1749042,Add Friend,25570,"Dallas-Fort Worth-Arlington, TX",4,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-10-02 08:00:26,2018-08-08 13:22:21
883667,307,M,paid,1749042,Thumbs Down,31304,"Dallas-Fort Worth-Arlington, TX",6,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-10-03 18:35:42,2018-08-08 13:22:21
893741,307,M,paid,1749042,Thumbs Down,31304,"Dallas-Fort Worth-Arlington, TX",18,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-10-03 19:10:47,2018-08-08 13:22:21
916990,307,M,paid,1749042,Thumbs Up,31304,"Dallas-Fort Worth-Arlington, TX",37,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-10-03 20:35:39,2018-08-08 13:22:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25654354,307,M,free,1152881,Thumbs Up,2537,"Greenville-Anderson-Mauldin, SC",25,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-11-19 21:28:17,2018-11-19 20:24:09
25654519,307,M,free,1152881,Add Friend,2537,"Greenville-Anderson-Mauldin, SC",28,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-11-19 21:31:26,2018-11-19 20:24:09
25654520,307,M,free,1152881,Add Friend,2537,"Greenville-Anderson-Mauldin, SC",29,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-11-19 21:31:27,2018-11-19 20:24:09
25660930,307,M,free,1152881,Thumbs Up,2537,"Greenville-Anderson-Mauldin, SC",61,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-11-19 23:44:56,2018-11-19 20:24:09


In [38]:
mask = (df_churn["status"] == 307) & (df_churn["length"] != "None")
len(df_churn[mask])

0

In [43]:
mask = df_churn["status"] == 307
unique_values_307 = df_churn[mask]["page"].unique()
print(unique_values_307)
unique_values_page = df_churn["page"].unique()
print(unique_values_page)

['Thumbs Up' 'Add Friend' 'Thumbs Down' 'Logout' 'Save Settings' 'Cancel'
 'Submit Downgrade' 'Submit Upgrade']
['NextSong' 'Downgrade' 'Help' 'Home' 'Thumbs Up' 'Add Friend'
 'Thumbs Down' 'Add to Playlist' 'Logout' 'About' 'Settings'
 'Save Settings' 'Cancel' 'Cancellation Confirmation' 'Submit Downgrade'
 'Roll Advert' 'Upgrade' 'Error' 'Submit Upgrade']


In [ ]:
mask = df_churn["status"] == 404
df_churn[mask]

,status,gender,level,userId,page,sessionId,location,itemInSession,userAgent,length,song,artist,time,registration
2788052,404,F,free,1697168,Error,58979,"Hilo, HI",14,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,None,None,None,2018-10-09 19:12:32,2018-09-08 13:48:25
13897750,404,F,paid,1697168,Error,175256,"Hilo, HI",37,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,None,None,None,2018-11-12 00:55:32,2018-09-08 13:48:25
16297984,404,F,paid,1697168,Error,201893,"Hilo, HI",318,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,None,None,None,2018-11-19 13:08:39,2018-09-08 13:48:25
602280,404,M,paid,1222580,Error,30295,"Watertown, SD",16,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,None,2018-10-02 22:06:26,2018-08-16 02:31:00
602568,404,M,paid,1222580,Error,30295,"Watertown, SD",18,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,None,2018-10-02 22:07:35,2018-08-16 02:31:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25087053,404,M,paid,1934047,Error,1109,"El Dorado, AR",75,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,None,2018-11-09 07:50:11,2018-08-31 04:28:43
25089645,404,M,paid,1934047,Error,1109,"El Dorado, AR",100,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,None,2018-11-09 09:19:53,2018-08-31 04:28:43
25179115,404,F,free,1912269,Error,2534,"Seattle-Tacoma-Bellevue, WA",72,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",None,None,None,2018-11-11 04:03:34,2018-11-11 01:12:59
25248788,404,M,free,1882230,Error,2951,"Houston-The Woodlands-Sugar Land, TX",98,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None,None,2018-11-12 17:33:20,2018-04-06 02:14:55


In [46]:
unique_values_404 = df_churn[mask]["page"].unique()
print(unique_values_404)

['Error']


In [47]:
mask = df_churn["status"] == 200
unique_values_200 = df_churn[mask]["page"].unique()
print(unique_values_200)

['NextSong' 'Downgrade' 'Help' 'Home' 'Add to Playlist' 'About' 'Settings'
 'Cancellation Confirmation' 'Roll Advert' 'Upgrade']


In [49]:
pages_200 = set(unique_values_200)
pages_307 = set(unique_values_307)

status_overlap = pages_200.intersection(pages_307)
if len(status_overlap) == 0:
    print("No overlap here. Safe to delete status")
else:
    print(f"Overlap in {status_overlap}. Can't delete status")


No overlap here. Safe to delete status


In [50]:
df_churn.drop(columns="status", inplace=True)

In [51]:
df_churn.sample(5)

,gender,level,userId,page,sessionId,location,itemInSession,userAgent,length,song,artist,time,registration
389116,M,paid,1665209,NextSong,3796,"Concord, NH",526,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",209.50159,Supermassive Black Hole (Twilight Soundtrack V...,Muse,2018-10-02 07:48:09,2018-09-19 16:52:10
25510001,F,paid,1778775,NextSong,32212,"San Diego-Carlsbad, CA",59,Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:1...,219.66322,Dog Days Are Over (Radio Edit),Florence + The Machine,2018-11-16 18:30:57,2018-08-24 03:39:47
5958205,F,free,1995049,NextSong,93460,"Johnson City, TN",7,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,220.96934,Heads Will Roll,Yeah Yeah Yeahs,2018-10-18 22:09:10,2018-09-28 23:31:21
11090483,M,paid,1379220,NextSong,148010,"Portland-Vancouver-Hillsboro, OR-WA",183,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",494.07955,Heroin,The Velvet Underground / Lou Reed,2018-11-02 20:11:28,2018-07-20 16:10:47
22160927,F,free,1519731,NextSong,21648,"Virginia Beach-Norfolk-Newport News, VA-NC",23,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,150.85669,Crack City Rockers,Leftover Crack,2018-11-01 12:51:57,2018-09-17 16:58:46


In [ ]:
location_split = df_churn['location'].str.split(', ', expand=True)

                                    0   1
0         Dallas-Fort Worth-Arlington  TX
992       Dallas-Fort Worth-Arlington  TX
1360      Dallas-Fort Worth-Arlington  TX
1825      Dallas-Fort Worth-Arlington  TX
2366      Dallas-Fort Worth-Arlington  TX
...                               ...  ..
25661184  Greenville-Anderson-Mauldin  SC
25661214  Greenville-Anderson-Mauldin  SC
25661320  Greenville-Anderson-Mauldin  SC
25661462  Greenville-Anderson-Mauldin  SC
25661583  Greenville-Anderson-Mauldin  SC

[17499636 rows x 2 columns]


In [54]:
df_churn['metropolitan_area'] = location_split[0].str.strip()
df_churn['state'] = location_split[1].str.strip()

In [55]:
nan_count_location = df_churn.isna().sum()
print(nan_count)

status                 0
gender                 0
level                  0
userId                 0
page                   0
sessionId              0
location               0
itemInSession          0
userAgent              0
length           3208203
song             3208203
artist           3208203
time                   0
registration           0
dtype: int64


In [56]:
df_churn['artist'] = df_churn['artist'].fillna("No artist")
df_churn['song'] = df_churn['song'].fillna("No song")
df_churn['length'] = df_churn['length'].fillna(0)

In [57]:
print(df_churn.isna().sum())

gender               0
level                0
userId               0
page                 0
sessionId            0
location             0
itemInSession        0
userAgent            0
length               0
song                 0
artist               0
time                 0
registration         0
metropolitan_area    0
state                0
dtype: int64


In [59]:
unique_states = df_churn["state"].unique()
print(len(unique_states))
unique_area = df_churn["metropolitan_area"].unique()
print(len(unique_area))
unique_location = df_churn["location"].unique()
print(len(unique_location))

100
806
875


In [60]:
print(unique_states)

['TX' 'CA' 'HI' 'SD' 'MD' 'FL' 'IN' 'PA' 'NY' 'VA' 'PA-NJ-DE-MD' 'VA-NC'
 'MA-NH' 'GA' 'KY' 'MO-IL' 'DC-VA-MD-WV' 'AZ' 'OH' 'OR' 'TN-MS-AR'
 'IL-IN-WI' 'TN-GA' 'NY-NJ-PA' 'MI' 'WA' 'WY' 'AL' 'NH' 'NV' 'SC' 'MN-WI'
 'TN' 'MN' 'CO' 'MO' 'OH-KY-IN' 'IN-KY' 'NE-IA' 'RI-MA' 'MA-CT' 'LA' 'CT'
 'OK' 'NC' 'NJ' 'WV' 'KS' 'MD-WV' 'MS' 'AR' 'PA-NJ' 'IL' 'WI' 'MS-LA' 'IA'
 'OR-WA' 'TN-VA' 'ME' 'NM' 'NC-SC' 'IA-IL' 'UT' 'KY-IN' 'GA-AL' 'MO-KS'
 'OH-PA' 'MA' 'NE' 'IN-MI' 'ID' 'AR-OK' 'TN-KY' 'SC-NC' 'MT' 'WV-KY-OH'
 'GA-SC' 'AR-MO' 'DE' 'ND' 'KY-IL' 'VT' 'AK' 'WY-ID' 'UT-ID' 'WV-OH'
 'TX-AR' 'MD-DE' 'IA-NE-SD' 'OR-ID' 'ND-MN' 'VA-WV' 'ID-WA' 'NH-VT'
 'IL-MO' 'WI-MI' 'IA-IL-MO' 'WI-MN' 'WV-VA' 'MI-WI']
